<br><br>
<font color='FireBrick' size='7'><b> Double/Debiased Machine Learning (DML)</b></font>

<font color='black' size='5'><b>using IPUMS data</b></font>

In [ ]:
import pandas as pd

main_data_df = pd.read_feather("/Users/dadmehr/R/main_data.feather")

print(f"Number of rows: {main_data_df.shape[0]}")
print(f"Number of columns: {main_data_df.shape[1]}")

In [ ]:
#data = data.sample(frac=0.01)  # Sampling 1% of the dataset due to GitHub's 25MB file size limit for uploads.

In [ ]:
data = main_data_df[main_data_df['YEAR'] == 2021]
data = data[['INCTOT', 'SEX', 'EDUC', 'EDUCD', 'RACE', 'AGE','EMPSTAT', 'MARST']]

data.to_csv('IPUMS_2021_data.csv', index=False)

In [ ]:
data

In [ ]:
#!pip install statsmodels econml

In [ ]:
import statsmodels.api as sm

# Prepare the variables
X = data[['SEX', 'AGE']]  # Independent variables
y = data['INCTOT']  # Dependent variable

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())


In [ ]:
import doubleml as dml
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load your data (replace 'your_data.csv' with your actual file name)

# Get the column names for the outcome, treatment, and control(s)
y_col = 'INCTOT'  # Outcome variable
d_col = 'SEX'  # Treatment variable
x_cols = ['AGE']  # Control variables

# Create the dataset for DoubleML
data_dml = dml.DoubleMLData(data, y_col=y_col, d_cols=[d_col], x_cols=x_cols)

# Initialize the machine learning algorithms for DML
ml_y = RandomForestRegressor()  # For the outcome variable
ml_d = RandomForestRegressor()  # For the treatment variable

# Initialize and fit the DoubleMLPLR model
dml_plr = dml.DoubleMLPLR(data_dml, ml_y, ml_d, n_folds=3)
dml_plr.fit()

# Retrieve the summary DataFrame
summary = dml_plr.summary

# Check the available column names
print(summary.columns)

# Extract the coefficient for the treatment variable (SEX)
sex_coef = summary['coef'].loc[d_col]

# Output the coefficient for SEX
print(f"The estimated coefficient for SEX is: {sex_coef}")